# 🚀 Image Classification Training on Google Colab (Free GPU)

**This notebook allows you to train your models with FREE GPU access!**

## Setup Instructions:
1. Go to: https://colab.research.google.com/
2. Upload this notebook (File → Upload notebook)
3. Enable GPU: Runtime → Change runtime type → GPU → Save
4. Run all cells!

**Training time: ~30 minutes with GPU vs 8+ hours on CPU!** ⚡

## Step 1: Setup Project Files

In [ ]:
# Check GPU availability
import torch
print(f"🔥 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  GPU not enabled! Go to Runtime → Change runtime type → GPU")

In [ ]:
# Install dependencies
!pip install -q timm albumentations opencv-python omegaconf hydra-core tqdm scikit-learn matplotlib seaborn

In [ ]:
# Create project structure
!mkdir -p configs/model configs/data configs/training
!mkdir -p src/data src/models src/training src/utils
!mkdir -p checkpoints logs outputs data

## Step 2: Upload Your Project Files

**Option A: Use Google Drive (Recommended)**
1. Upload your entire `image_classification` folder to Google Drive
2. Run the cell below to mount Drive
3. Copy files from Drive

**Option B: Upload Manually**
- Click the folder icon on the left
- Upload your `src/` folder and `configs/` folder

**Option C: Copy-paste code (see cells below)**

In [ ]:
# Option A: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy your project from Drive (adjust path as needed)
# !cp -r /content/drive/MyDrive/image_classification/* /content/

## Step 3: Quick Setup (Copy-Paste Your Code)

**If you don't want to upload files, I'll create minimal code here:**

In [ ]:
%%writefile src/models/simple_model.py
import torch.nn as nn
import timm

def create_model(model_name='resnet50', num_classes=10, pretrained=True):
    """Create model using timm."""
    model = timm.create_model(
        model_name,
        pretrained=pretrained,
        num_classes=num_classes
    )
    return model

In [ ]:
%%writefile train_simple.py
"""Simplified training script for Colab."""
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import sys
sys.path.insert(0, '/content')
from src.models.simple_model import create_model

# Configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 64 if DEVICE == 'cuda' else 32
EPOCHS = 10
LR = 0.001

print(f"Training on: {DEVICE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}\n")

# Data
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}\n")

# Model
model = create_model('resnet50', num_classes=10, pretrained=True).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Training loop
best_acc = 0
for epoch in range(EPOCHS):
    # Train
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}')
    for images, labels in pbar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'loss': f'{train_loss/len(pbar):.4f}', 'acc': f'{100.*train_correct/train_total:.2f}%'})
    
    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_acc = 100. * val_correct / val_total
    scheduler.step()
    
    print(f'Epoch {epoch+1}: Train Acc: {100.*train_correct/train_total:.2f}% | Val Acc: {val_acc:.2f}%')
    
    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'accuracy': val_acc
        }, 'best_model.pth')
        print(f'✓ Saved best model (acc: {val_acc:.2f}%)')

print(f'\n🎉 Training complete! Best accuracy: {best_acc:.2f}%')

## Step 4: Train! 🚀

In [ ]:
# Run training
!python train_simple.py

## Step 5: Download Your Trained Model

In [ ]:
# Download the trained model
from google.colab import files
files.download('best_model.pth')

## Alternative: Use Your Full Project

If you uploaded your complete project files:

In [ ]:
# If you have the full project structure, run:
# !python scripts/train.py training.epochs=10 data.batch_size=64 device=cuda

## Try Different Models

In [ ]:
# Vision Transformer
model_vit = create_model('vit_base_patch16_224', num_classes=10, pretrained=True)
print(f"ViT parameters: {sum(p.numel() for p in model_vit.parameters()):,}")

# EfficientNet
model_eff = create_model('efficientnet_b3', num_classes=10, pretrained=True)
print(f"EfficientNet parameters: {sum(p.numel() for p in model_eff.parameters()):,}")

## Expected Results (10 epochs)

- **ResNet50**: ~94% validation accuracy in ~30 minutes
- **ViT-Base**: ~95% validation accuracy in ~60 minutes  
- **EfficientNet-B3**: ~96% validation accuracy in ~45 minutes

## Tips:

1. **Check GPU**: Make sure GPU is enabled (see cell 1)
2. **Session limit**: Colab free tier has ~12 hour limit
3. **Save often**: Download your model periodically
4. **Reconnect**: If disconnected, just re-run from training cell

## Next Steps:

- Try different models (ViT, EfficientNet)
- Increase epochs to 50-100 for better results
- Experiment with batch size (32, 64, 128)
- Add more augmentations

**Enjoy your FREE GPU training! 🎉**